# Collective movement

### Using `EvolvingClustersKDT.py` from the `DataStories-UniPi/EvolvingClusters` repository

In [1]:
from EvolvingClustersKDT import evolving_clusters

### Imports

In [4]:
from sklearn.cluster import KMeans
from matplotlib import pyplot as plt
from sqlalchemy import create_engine
import geopandas as gpd
import numpy as np
import psycopg2

from config import DB_CONFIG
from preprocess import cleanse

### Establish a db connection with PostGIS and cleanse the data

In [11]:
RECORDS_LIMIT = 50000
conn = psycopg2.connect(**DB_CONFIG)

all_points_sql = f'SELECT * FROM vessels_points_jan LIMIT {RECORDS_LIMIT} ORDER BY timest ASC'
all_points_gdf =  gpd.GeoDataFrame.from_postgis(all_points_sql, conn, geom_col='geom')
gdf = cleanse(all_points_gdf)

gdf.head()

,timest,dates,typee,mmsi,status,heading,turn,speed,course,geom,acceleration
0,1.514824e+12,2018-01-01 16:32:09,None,239611000,None,105.811638,None,0.008879,107.8,POINT (23.57052 37.95392),-3.160223e-09
1,1.514824e+12,2018-01-01 16:32:09,None,248287000,None,0.000000,None,0.000000,236.2,POINT (23.66830 37.93847),0.000000e+00
2,1.514824e+12,2018-01-01 16:32:09,None,239634000,None,322.909179,None,0.007658,341.8,POINT (23.60115 37.94287),-1.623614e-08
3,1.514824e+12,2018-01-01 16:32:10,None,636017265,None,122.463934,None,0.002763,132.0,POINT (23.59912 37.88728),-2.387838e-08
4,1.514824e+12,2018-01-01 16:32:10,None,237611000,None,0.000000,None,0.000000,330.8,POINT (23.63504 37.94351),-5.881296e-09


In [18]:
gdf['lat'] = gdf['geom'].apply(lambda point: point.x)
gdf['lon'] = gdf['geom'].apply(lambda point: point.y)
gdf['timest'] = gdf['timest'].apply(lambda t: t / 1000.0)
gdf.head()

,timest,dates,typee,mmsi,status,heading,turn,speed,course,geom,acceleration,lon,lat
0,1.514824e+09,2018-01-01 16:32:09,None,239611000,None,105.811638,None,0.008879,107.8,POINT (23.57052 37.95392),-3.160223e-09,37.953917,23.570517
1,1.514824e+09,2018-01-01 16:32:09,None,248287000,None,0.000000,None,0.000000,236.2,POINT (23.66830 37.93847),0.000000e+00,37.938467,23.668300
2,1.514824e+09,2018-01-01 16:32:09,None,239634000,None,322.909179,None,0.007658,341.8,POINT (23.60115 37.94287),-1.623614e-08,37.942870,23.601147
3,1.514824e+09,2018-01-01 16:32:10,None,636017265,None,122.463934,None,0.002763,132.0,POINT (23.59912 37.88728),-2.387838e-08,37.887283,23.599117
4,1.514824e+09,2018-01-01 16:32:10,None,237611000,None,0.000000,None,0.000000,330.8,POINT (23.63504 37.94351),-5.881296e-09,37.943513,23.635040


In [19]:
res_mcs, res_mc = evolving_clusters(gdf, distance_threshold=1500, min_cardinality=5, time_threshold=5, disable_progress_bar=False, temporal_name='timest')

100%|██████████| 18556/18556 [04:40<00:00, 66.23it/s] 


In [24]:
print(res_mc)
print(res_mcs)

Empty DataFrame
Columns: [clusters, st, et]
Index: []
Empty DataFrame
Columns: [clusters, st, et]
Index: []
